In [24]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

#### Homework Reflection 9

1. Write some code that will use a simulation to estimate the standard deviation of the coefficient when there is heteroskedasticity.  Compare these standard errors to those found via statsmodels OLS or a similar linear regression model.

In [ ]:
np.random.seed(42)

n = 100
n_simulations = 1000
beta_true = 2.0
intercept = 1.0

X = np.random.normal(0, 1, n)
X_with_const = sm.add_constant(X)

# Function to simulate heteroskedasticity errors
def generate_heteroskedastic_errors(X, scale = 1.0):
    sigma = scale * (1 + np.abs(X)) # error variance increases with X
    errors = np.random.normal(0, sigma, len(X))
    return errors

# Data simulation and coefficient estimation
beta_estimates = []
for _ in range(n_simulations):
    errors = generate_heteroskedastic_errors(X)
    y = intercept + beta_true * X + errors
    model = sm.OLS(y, X_with_const).fit()
    beta_estimates.append(model.params[1])

# Simulation based standard deviation of the coefficient
beta_estimates = np.array(beta_estimates)
sim_std = np.std(beta_estimates, ddof = 1)

# Fit OLS model on one dataset to get standard errors
errors = generate_heteroskedastic_errors(X)
y = intercept + beta_true * X + errors
model = sm.OLS(y, X_with_const).fit()
ols_se = model.bse[1]  

print(f"Simulation-based standard deviation of coefficient: {sim_std:.4f}")
print(f"OLS standard error of coefficient: {ols_se:.4f}")

Simulation-based standard deviation of coefficient: 0.2825
OLS standard error of coefficient: 0.1915


<font color = 'seagreen'> The standard error of the coefficient found from simulation with heteroskedasticity was 0.2825. This is a larger value than the standard error of the coefficient found via statsmodels OLS, which was 0.1915. This shows that OLS standard errors are unreliable under heteroskedasticity and that simulation-based approaches better reflect the true uncertainties in the coefficient estimates.

2. Write some code that will use a simulation to estimate the standard deviation of the coefficient when errors are highly correlated / non-independent.
Compare these standard errors to those found via statsmodels OLS or a similar linear regression model.

In [3]:
np.random.seed(42)

n = 100
n_simulations = 1000
beta_true = 2.0
intercept = 1.0
rho = 0.9
sigma = 1.0

X = np.random.normal(0, 1, n)
X_with_const = sm.add_constant(X)

# Function to generate correlated errors
def generate_correlated_errors(n, rho, sigma):
    errors = np.zeros(n)
    errors[0] = np.random.normal(0, sigma)
    for t in range(1, n):
        errors[t] = rho * errors[t-1] + np.random.normal(0, sigma * np.sqrt(1 - rho**2))
    return errors

# Data simulation and coefficient estimation
beta_estimates = []
for _ in range(n_simulations):
    errors = generate_correlated_errors(n, rho, sigma)
    y = intercept + beta_true * X + errors
    model = sm.OLS(y, X_with_const).fit()
    beta_estimates.append(model.params[1])

# Calculate simulation based std
beta_estimates = np.array(beta_estimates)
sim_std = np.std(beta_estimates, ddof = 1)

# Fit OLS model on one dataset to get standard errors
errors = generate_correlated_errors(n, rho, sigma)
y = intercept + beta_true * X + errors
model = sm.OLS(y, X_with_const).fit()
ols_se = model.bse[1]

print(f"Simulation-based standard deviation of coefficient (correlated errors): {sim_std:.4f}")
print(f"OLS standard error of coefficient (correlated errors): {ols_se:.4f}")

Simulation-based standard deviation of coefficient (correlated errors): 0.1033
OLS standard error of coefficient (correlated errors): 0.0717


<font color='seagreen'> Again, the simulation based standard deviation (0.1033) was larger than the OLS standard error of 0.0717. This is likely because OLS assumptions are violated when errors are highly correlated. The simulated data is also able to capture true variability because the simulation-based standard deviation is derived from repeatedly sampling data with correlated errors and estimating the coefficient. Overall, this indicates that the true uncertainty in the coefficient estimates is greater than what OLS assumes due to error correlation.

Show that if the correlation between coefficients is high enough, then the estimated standard deviation of the coefficient, using bootstrap errors, might not match that found by a full simulation of the Data Generating Process. (This can be fixed if you have a huge amount of data for the bootstrap simulation.)

In [7]:
np.random.seed(42)

def run_simulation(n, n_simulations = 1000, n_boostrap = 1000, rho_pred = 0.95, rho_err = 0.9, sigma = 1.0):
    beta_true = np.array([1.0, 2.0])
    intercept = 1.0
    
    cov_matrix = np.array([[1.0, rho_pred], [rho_pred, 1.0]])
    
    def generate_correlated_errors(n, rho, sigma):
        errors = np.zeros(n)
        errors[0] = np.random.normal(0, sigma)
        for t in range(1, n):
            errors[t] = rho * errors[t-1] + np.random.normal(0, sigma * np.sqrt(1 - rho**2))
        return errors
    
    # Full DGP simulation
    beta1_dgp_estimates = []
    for _ in range(n_simulations):
        X = np.random.multivariate_normal(mean = [0, 0], cov = cov_matrix, size = n)
        X_with_const = sm.add_constant(X)
        errors = generate_correlated_errors(n, rho_err, sigma)
        y = intercept + X.dot(beta_true) + errors
        model = sm.OLS(y, X_with_const).fit()
        beta1_dgp_estimates.append(model.params[1])
    
    dgp_std = np.std(beta1_dgp_estimates, ddof = 1)

    # Generate one dataset for bootstrap and OLS
    X = np.random.multivariate_normal(mean = [0, 0], cov = cov_matrix, size = n)
    X_with_const = sm.add_constant(X)
    errors = generate_correlated_errors(n, rho_err, sigma)
    y = intercept + X.dot(beta_true) + errors
    model = sm.OLS(y, X_with_const).fit()
    ols_se = model.bse[1]
    
    # Bootstrap simulation
    fitted_values = model.fittedvalues
    residuals = model.resid
    beta1_bootstrap_estimates = []
    for _ in range(n_boostrap):
        resampled_residuals = np.random.choice(residuals, size = n, replace = True)
        y_bootstrap = fitted_values + resampled_residuals
        model_bootstrap = sm.OLS(y_bootstrap, X_with_const).fit()
        beta1_bootstrap_estimates.append(model_bootstrap.params[1])
    
    boot_std = np.std(beta1_bootstrap_estimates, ddof = 1)
    
    return dgp_std, ols_se, boot_std, beta1_dgp_estimates, beta1_bootstrap_estimates

n_small = 100
dgp_std_small, ols_se_small, boot_std_small, dgp_estimates_small, boot_estimates_small = run_simulation(n_small)
print(f"Small Sample Size (n={n_small}):")
print(f"  Full DGP Simulation Std: {dgp_std_small:.4f}")
print(f"  OLS Std Error: {ols_se_small:.4f}")
print(f"  Bootstrap Std: {boot_std_small:.4f}")

n_large = 10000
dgp_std_large, ols_se_large, boot_std_large, dgp_estimates_large, boot_estimates_large = run_simulation(n_large)
print(f"\nLarge Sample Size (n={n_large}):")
print(f"  Full DGP Simulation Std: {dgp_std_large:.4f}")
print(f"  OLS Std Error: {ols_se_large:.4f}")
print(f"  Bootstrap Std: {boot_std_large:.4f}")

Small Sample Size (n=100):
  Full DGP Simulation Std: 0.2968
  OLS Std Error: 0.2389
  Bootstrap Std: 0.2293

Large Sample Size (n=10000):
  Full DGP Simulation Std: 0.0320
  OLS Std Error: 0.0310
  Bootstrap Std: 0.0318


<font color='seagreen'> Using bootstrap errors, it's seen that the estimated standard deviation of the coefficient doesn't make the standard deviation found using a full simulation of the Data Generating Process. From the small sample size data, the bootstrap standard deviation found was 0.2293 and the full DGP simulation standard deviation found was 0.2968. However, as mentioned, this was fixed after using a larger sample data for the bootstrap simulation. So, the bootstrap standard deviation from the larger sample was 0.0318 and the full DGP simulation standard deviation was 0.0320.

#### Homework Reflection 11

1. Construct a dataset for an event study where the value, derivative, and second derivative for a trend all change discontinuously (suddenly) after an event. Build a model that tries to decided whether the event is real (has a nonzero effect) using:  
(a) only the value,  
(b) the value, derivative, and second derivative.  
Which of these models is better at detecting and/or quantifying the impact of the event? (What might "better" mean here?)

In [10]:
# Generate dataset
def generate_data(n = 200, sigma = 1.0, level_jump = 1.0, slope_change = 0.5, curv_change = 0.02):
    t_left = np.linspace(-5, 0, n, endpoint=False)
    t_right = np.linspace(0, 5, n, endpoint=False)
    t = np.concatenate([t_left, t_right])
    post = (t > 0).astype(float)
    y_left = 0 + 0.5 * t_left + 0.01 * t_left ** 2
    y_right = level_jump + (0.5 + slope_change) * t_right + (0.01 + curv_change) * t_right ** 2
    y_det = np.concatenate([y_left, y_right])
    errors = np.random.normal(0, sigma, len(t))
    y = y_det + errors
    return y, t, post

# Fit model a
def fit_model_a(y, t, post):
    X_a = np.column_stack((np.ones(len(t)), t, t**2, post))
    model_a = sm.OLS(y, X_a).fit()
    print(model_a.summary())
    p_value_a = model_a.pvalues[3]
    delta0_a = model_a.params[3]
    return p_value_a, delta0_a

# Fit model b
def fit_model_b(y, t, post):
    post_t = post * t
    post_t2 = post * t**2
    X_b = np.column_stack((np.ones(len(t)), t, t**2, post, post_t, post_t2))
    model_b = sm.OLS(y, X_b).fit()
    print(model_b.summary())
    R = np.array(([0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]))
    f_test = model_b.f_test(R)
    p_value_b = f_test.pvalue
    delta0_b = model_b.params[3]
    return p_value_b, delta0_b

# Generate and fit dataset
np.random.seed(42)
y, t, post = generate_data()
p_a, delta0_a = fit_model_a(y, t, post)
p_b, delta0_b = fit_model_b(y, t, post)
print(f"Example fit (a): p-value = {p_a: .4f}, estimated level jump = {delta0_a: .4f}")
print(f"Example fit (b): p-value = {p_b: .4f}, estimated level jump = {delta0_b: .4f}")

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     1141.
Date:                Sat, 09 Aug 2025   Prob (F-statistic):          2.12e-194
Time:                        10:09:46   Log-Likelihood:                -551.02
No. Observations:                 400   AIC:                             1110.
Df Residuals:                     396   BIC:                             1126.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3382      0.120      2.819      0.0

<font color='seagreen'> After creating a dataset and modeled it under the two scenarios, I found that model B is the "better" model at detecting and quantifying the impact. Model B is generally better for both detection and quantification in realistic event  studies. It correctly specifies the data-generating process, allowing it to detect subtle effects in derivatives/curvatures and provides unbiased estimates of the impact. In practice, "better" depends on the context: if computational simplicity matters and only level jumps are expected, model A suffices; otherwise, model B is preferable for robust inference.

2. Construct a dataset in which there are three groups whose values each increase discontinuously (suddenly) by the same amount at a shared event; they change in parallel over time, but they have different starting values. Create a model that combines group fixed effects with an event study, as suggested in the online reading. Explain what you did, how the model works, and how it accounts for both baseline differences and the common event effect.

In [26]:
np.random.seed(42)

groups = ['A', 'B', 'C']
baselines = {'A': 20, 'B': 30, 'C': 40}
time_periods = range(1, 11)
event_time = 5
jump = 10
trend_slope = 1

data = []
for group in groups:
    for t in time_periods:
        y = baselines[group] + (t - 1) * trend_slope
        if t >= event_time:
            y += jump
        post = 1 if t >= event_time else 0
        
        data.append({'group': group, 'time': t, 'post': post, 'y': y})

df = pd.DataFrame(data)

model = smf.ols('y ~ C(group) + post + time', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.976e+29
Date:                Sat, 09 Aug 2025   Prob (F-statistic):               0.00
Time:                        10:46:29   Log-Likelihood:                 869.61
No. Observations:                  30   AIC:                            -1729.
Df Residuals:                      25   BIC:                            -1722.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        19.0000   3.36e-14   5.65e+14

<font color='seagreen'> I constructed a synthetic panel dataset with 3 groups (A, B, C) observed over 10 time periods. Each group has a baseline (A: 20, B: 30, C: 40) and a linear trend of 1 unit per period. The event occurred at t = 5 where a 10-unit jump happened. The groups have different baseline levels (0, 5, 10 for groups 1, 2, 3, respectively), but they evolve in parallel where each has the same linear trend (slope = 0.5) before and after the event. There was no change in the trend at the event, only a level shift.  
The C(group) term creates dummies for groups B and C, capturing their baseline differences relative to Group A. This ensures that the model accounts for the different starting values, isolating the event effect from these constant differences. The post variable estimates the common jump across all groups at t = 5. The coefficient of 10 reflects the 10-unit increase, applied equally to all groups. The time variable controls for the linear trend (1 unit per period), ensuring the event is not confounded with the gradual increase over time. The model assumes parallel trends across groups, which our data satisfies. The fixed effects and time trend ensure the jump is attributed to the event, not baseline differences or trends.  
The group fixed effects absorb the constant differences in levels. This prevents baseline differences from biasing the event effect estimate. The post coefficient captures the shared 10-unit jump at t = 5, net of baseline differences and trends. Because the model includes group fixed effects, it isolates the event's impact across all groups. The parallel trends assumption is satisfied, where all groups have the same slope before and after the event. The model correctly attributes the jump to the event, not to divergent trends. </font>

#### Homework Reflection 12

Construct a dataset in which prior trends do not hold, and in which this makes the differences-in-differences come out wrong. Explain why the differences-in-differences estimate of the effect comes out higher or lower than the actual effect.

In [29]:
data = {
    'Unit_ID': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4],
    'Group': ['Control', 'Control', 'Control', 'Control', 'Control', 'Control', 'Treatment', 'Treatment', 'Treatment', 'Treatment', 'Treatment', 'Treatment'],
    'Year': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3],
    'Income': [100, 100, 100, 100, 100, 100, 100, 110, 125, 100, 110, 125]
}

df = pd.DataFrame(data)
df

,Unit_ID,Group,Year,Income
0,1,Control,1,100
1,1,Control,2,100
2,1,Control,3,100
3,2,Control,1,100
4,2,Control,2,100
5,2,Control,3,100
6,3,Treatment,1,100
7,3,Treatment,2,110
8,3,Treatment,3,125
9,4,Treatment,1,100


<font color='seagreen'> From this basic dataset example, the average pre-treatment income (Years 1-2) has a control of 100 and treatment of 105. Post-treatment income in Year 3 has a control of 100 and treatment of 125. The change for the control is 100 - 100 = 0 while the change for treatment is 125 - 105 = +20. Thus, the DiD estimate would be +20 - 0 = +20. However, the actual treatment effect is +5 (observed 125 minus counterfactual 120 for the treatment group in Year 3). The DiD estimate of +20 is higher than the true effect by +15. This bias occurs because the parallel trends assumption is violated: in the absence of treatment, the treatment group would not have followed the same trend as the control group. The treatment group has a pre-existing upward trend (+10 from Year 1 to 2, and would have continued +10 from Year 2 to 3 without treatment), while the control group is flat. The DiD method assumes that any differential change post-treatment is solely due to the treatment, but here it incorrectly attributes the treatment group's ongoing upward trend (+10 from Year 2 to 3, plus the prior differential embedded in the pre-average) to the treatment effect. As a result, the estimate captures both the true effect of +5 and the extraneous trend of +15, leading to an overestimate.